# Insert/update data within the Wikibase

### Example entry to wikibase
This script will use the newly created `wikibasedataconnector` library to connect to the wikibase.

Since this is a wrapper class for the pywikibot library, the connection is established in the same way that was done using pywikibot.

Once that has been established, it will take the json template and load a mine entry into it. It will also find the correct item by the mrds id if it already exists in the wikibase.

In [1]:
import json
import sys
import os
import csv
import asyncio
from wikibasedataconnector import WBDC
from dotenv import load_dotenv

load_dotenv()

True

The method of processing the data will be provided in a conf file that can be passed into the `WBDC` bot, that will handle the different situations the conf files provide.

Lets assume the following
* The GeoKB has just been generated and has no data imported into it
* The set up and connection to the GeoKB and the WBDC bot has been established

First we load the property data. This data can be found at `data/property.csv` within the repo.

In [2]:
conf_filepath = '../../scripts/conf'
site_name = 'mrdata'

In [3]:
PATH = f'{conf_filepath}/import_properties_add.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

With the conf file mapping the indeces found within the csv file, the properties can now be loaded.

In [4]:
bot = WBDC(site_name)
bot.set_mapping_config(mapping_config)

FILEPATH = '../../data/properties.csv'
with open(FILEPATH, 'r', encoding='utf-8') as file:
    csv_file = csv.reader(file)
    next(csv_file)
    await asyncio.gather(*[bot.process(row) for row in csv_file])

{'entity': {'type': 'property', 'datatype': 'wikibase-item', 'id': 'P1', 'labels': {'en': {'language': 'en', 'value': 'instance of'}}, 'descriptions': {'en': {'language': 'en', 'value': 'this item is a class of that item;type of the item'}}, 'aliases': {}, 'claims': {}, 'lastrevid': 2}, 'success': 1}
{'entity': {'type': 'property', 'datatype': 'wikibase-item', 'id': 'P2', 'labels': {'en': {'language': 'en', 'value': 'subclass of'}}, 'descriptions': {'en': {'language': 'en', 'value': 'this item is a subclass (subset) of that item'}}, 'aliases': {}, 'claims': {}, 'lastrevid': 3}, 'success': 1}
{'entity': {'type': 'property', 'datatype': 'globe-coordinate', 'id': 'P3', 'labels': {'en': {'language': 'en', 'value': 'coordinate location'}}, 'descriptions': {'en': {'language': 'en', 'value': "a geographic point indicating the location of an entity on the earth's surface"}}, 'aliases': {}, 'claims': {}, 'lastrevid': 4}, 'success': 1}
{'entity': {'type': 'property', 'datatype': 'wikibase-item',

Once that has been processed, we can load the prerequisite items such as country, states and subclass information needed (this is found at `data/items/csv`).

In [5]:
PATH = f'{conf_filepath}/import_items_add.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

In [7]:
bot = WBDC(site_name)
conf = {
    'uniqueIDProp': 'P13',
    'SPARQL_ENDPOINT': os.getenv('SPARQL_ENDPOINT')
}
bot.set_config(conf)
bot.set_mapping_config(mapping_config)

FILEPATH = '../../data/items.csv'
with open(FILEPATH, 'r', encoding='utf-8') as file:
    csv_file = csv.reader(file)
    next(csv_file)
    await asyncio.gather(*[bot.process(row) for row in csv_file])

Now that we have all the prior properties and items needed, all we need to do is process the mines data. For now a subset of MRDS will be used to demonstrate this. 

The way the data is processed is by indexing the entry of the necessary fields. This has the advantage of having variable data entries, and the fact that various data files can be read and processed. This will all be given within each provided conf file, which can be generated beforehand and passed in to the WBDC bot.

In [8]:
PATH = f'{conf_filepath}/import_mrds_conf.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

In [9]:
bot = WBDC(site_name)
conf = {
    'uniqueIDProp': 'P13',
    'SPARQL_ENDPOINT': os.getenv('SPARQL_ENDPOINT')
}
bot.set_config(conf)
bot.set_mapping_config(mapping_config)

FILEPATH = '../../data/mrds_subset.csv'
with open(FILEPATH, 'r', encoding='utf-8') as file:
    csv_file = csv.reader(file)
    next(csv_file)
    await asyncio.gather(*[bot.process(row) for row in csv_file])

### Reading From and API Endpoint

Another method to retrieve data is from an API which can then be processed into the wikibase. To show this, the counties dataset found at `https://data.colorado.gov/resource/ahgn-r8s5` will be used.

In [10]:
import requests

In [11]:
req = requests.get('https://data.colorado.gov/resource/ahgn-r8s5.json')
res = req.json()

Because there isn't any metadata found in the dataset explaining what the fields are, a workaround was created. Each field value that is not `county` will be added in as a property to use for mapping statements (for now `the_geom` field will be excluded since it uses a shape datatype, which makes the example more complicated than needed).

In [12]:
keys = list(res[0].keys())
keys = [key for key in keys if key not in ('county', 'the_geom')]
description = ['test description' for i in keys]
datatypes = []
for key in keys:
    if key == 'the_geom':
        datatypes.append('geo-shape')
    else:
        datatypes.append('string')

counties_props = list(zip(keys,description,datatypes))

In [13]:
counties_props

[('co_fips', 'test description', 'string'),
 ('pop_2010', 'test description', 'string'),
 ('househo_20', 'test description', 'string'),
 ('shape_star', 'test description', 'string'),
 ('shape_stle', 'test description', 'string'),
 ('shape_st_1', 'test description', 'string'),
 ('shape_st_2', 'test description', 'string')]

In [14]:
PATH = f'{conf_filepath}/import_counties_properties.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

In [15]:
bot = WBDC(site_name)
bot.set_mapping_config(mapping_config)

await asyncio.gather(*[bot.process(row) for row in counties_props])

{'entity': {'type': 'property', 'datatype': 'string', 'id': 'P15', 'labels': {'en': {'language': 'en', 'value': 'co_fips'}}, 'descriptions': {'en': {'language': 'en', 'value': 'test description'}}, 'aliases': {}, 'claims': {}, 'lastrevid': 169}, 'success': 1}
{'entity': {'type': 'property', 'datatype': 'string', 'id': 'P16', 'labels': {'en': {'language': 'en', 'value': 'pop_2010'}}, 'descriptions': {'en': {'language': 'en', 'value': 'test description'}}, 'aliases': {}, 'claims': {}, 'lastrevid': 170}, 'success': 1}
{'entity': {'type': 'property', 'datatype': 'string', 'id': 'P17', 'labels': {'en': {'language': 'en', 'value': 'househo_20'}}, 'descriptions': {'en': {'language': 'en', 'value': 'test description'}}, 'aliases': {}, 'claims': {}, 'lastrevid': 171}, 'success': 1}
{'entity': {'type': 'property', 'datatype': 'string', 'id': 'P18', 'labels': {'en': {'language': 'en', 'value': 'shape_star'}}, 'descriptions': {'en': {'language': 'en', 'value': 'test description'}}, 'aliases': {}, 

[None, None, None, None, None, None, None]

Now that the fields are added as properties we can load the county data using the conf file. 

Notice that the conf file uses the field names as indeces. This is because indexing a dictionary with the key and indexing an array by an integer is the same syntax in Python.

In [18]:
PATH = f'{conf_filepath}/import_counties.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

In [19]:
bot = WBDC(site_name)
conf = {
    'uniqueIDProp': 'P13',
    'SPARQL_ENDPOINT': os.getenv('SPARQL_ENDPOINT')
}
bot.set_config(conf)
bot.set_mapping_config(mapping_config)
await asyncio.gather(*[bot.process(row) for row in res])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Adding a single claim

This example shows how to insert a statement into the desired data found within the wikibase. The subset of MRDS will be used, since it was already loaded in an earlier example found in this notebook.

In [20]:
PATH = f'{conf_filepath}/import_mrds_add_claim.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

In [21]:
bot = WBDC(site_name)
conf = {
    'uniqueIDProp': 'P13',
    'SPARQL_ENDPOINT': os.getenv('SPARQL_ENDPOINT')
}
bot.set_config(conf)
bot.set_mapping_config(mapping_config)

FILEPATH = '../../data/mrds_subset.csv'
with open(FILEPATH, 'r', encoding='utf-8') as file:
    csv_file = csv.reader(file)
    next(csv_file)
    await asyncio.gather(*[bot.process(row) for row in csv_file])

### Retrieving Existing Items in GeoKB and Updating Claims

Suppose that there is some existing item pages within the wikibase that you want to update with new information. A way to do this would be to get the items of interest and use the config file to indicate what new information will be inserted. 

These two examples show two approaches to inserting the new data:
* Use the results given by the SPARQL query to insert a literal value found in the mapping config file
* Append another column of new information to the SPARQL query results using Pandas and process the new column by the index

Before we can show how to process the data, we first need to extract the items of interest. To do this, the following needs to be set.

In [22]:
sparql_endpoint = os.getenv('SPARQL_ENDPOINT')
unique_id_prop = 'P13'

query = """
SELECT DISTINCT ?item ?itemLabel ?itemDescription ?itemAltLabel WHERE 
{
  ?item wdt:"""+unique_id_prop+""" ?uniqueid  . 
   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

LIMIT 20
"""
print(query)



SELECT DISTINCT ?item ?itemLabel ?itemDescription ?itemAltLabel WHERE 
{
  ?item wdt:P13 ?uniqueid  . 
   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

LIMIT 20



For ease of use, the SPARQLWrapper library is used to send the query to the endpoint. This has the advantage of parsing out a multiple line SPARQL query, which throws an error when using the standard `requests` library.

In [23]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(sparql_endpoint)

sparql.setReturnFormat(JSON)
sparql.setQuery(query)
res = sparql.queryAndConvert()


Now that the response is successful, the JSON will be parsed into a tabular format for processing.

In [24]:
table = []
bindings = res['results']['bindings']
header = list(bindings[0].keys())
for binding in bindings:
    row = []
    for _, val in binding.items():
        row.append(val['value'])
    table.append(row)


This first approach will be to add a literal value specified within the mapping config file.

In [25]:
bot = WBDC(site_name)

conf = {
    'uniqueIDProp': unique_id_prop,
    'SPARQL_ENDPOINT': sparql_endpoint
}
bot.set_config(conf)


In [26]:
PATH = f'{conf_filepath}/import_claim_with_sparql_literal.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)
        bot.set_mapping_config(mapping_config)
        await asyncio.gather(*[bot.process(row) for row in table])

Another method that can be used is to utilize Pandas to append new columns with their corresponding information, and process that back into the GeoKB by indexing the new columns within the mapping config file.

In [27]:
import pandas as pd
df = pd.DataFrame(table, columns=header)

In [28]:
df['added_claim'] = 'this is an added statement'

In [29]:
df

,item,itemLabel,itemDescription,itemAltLabel,added_claim
0,http://wiki.demo5280.com/entity/Q57,Lookout Prospect,this is the desc,"Conundrum, Mammoth, Wakefield Minerals Co.",this is an added statement
1,http://wiki.demo5280.com/entity/Q62,Marion Prospect,this is the desc,Nutqua Gold Mining Co.,this is an added statement
2,http://wiki.demo5280.com/entity/Q59,LAKE,this is the desc,"Claims: Horseshoe, Copper, Lake Bay",this is an added statement
3,http://wiki.demo5280.com/entity/Q58,Lucky Find Prospect,this is the desc,None,this is an added statement
4,http://wiki.demo5280.com/entity/Q63,Marble Heart Prospect,this is the desc,None,this is an added statement
5,http://wiki.demo5280.com/entity/Q60,Lucky Jim Claim,this is the desc,None,this is an added statement
6,http://wiki.demo5280.com/entity/Q61,Matilda Occurrence,this is the desc,None,this is an added statement
7,http://wiki.demo5280.com/entity/Q66,Miller Brothers Claim,this is the desc,None,this is an added statement
8,http://wiki.demo5280.com/entity/Q64,Morning Star Prospect,this is the desc,None,this is an added statement
9,http://wiki.demo5280.com/entity/Q65,Monday Prospect,this is the desc,None,this is an added statement


In [30]:
PATH = f'{conf_filepath}/import_claim_with_sparql.json'
with open(PATH, 'r', encoding='utf-8') as f:
        mapping_config = json.load(f)

In [31]:
bot = WBDC(site_name)
conf = {
    'uniqueIDProp': unique_id_prop,
    'SPARQL_ENDPOINT': sparql_endpoint
}
bot.set_config(conf)
bot.set_mapping_config(mapping_config)


Since a DataFrame is being passed in this example, a different approach is used since usually Pandas doesn't recommend iterating rows.

In [32]:
await asyncio.gather(*df.apply(lambda df_row: bot.process(df_row), raw=True, axis=1))

[None, None, None, None, None, None, None, None, None, None]